In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['ARF']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('acute renal failure', 559),
 ('adp ribosylation factor', 266),
 ('auxin response factor', 121),
 ('acute respiratory failure', 105),
 ('alternative reading frame', 72),
 ('acute rheumatic fever', 48),
 ('acoustic radiation force', 38),
 ('adventitious root formation', 9),
 ('arf', 7),
 ('alkaloid rich fraction', 7),
 ('alternative reading frame protein', 6),
 ('adpribosylation factor', 6),
 ('anthocyanin rich fraction', 3),
 ('alternative open reading frame', 3),
 ('adenosine diphosphate ribosylation factor', 3)]

In [6]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [7]:
names

{'MESH:D058186': 'Acute Kidney Injury',
 'FPLX:ARF_GTPase_family': 'ARF_GTPase_family',
 'MESH:D012131': 'Respiratory Insufficiency',
 'HGNC:1787': 'CDKN2A',
 'MESH:D012213': 'Rheumatic Fever',
 'MESH:D016366': 'Open Reading Frames',
 'HGNC:30833': 'CDK2AP2'}

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-11 05:07:29] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [9]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'acoustic radiation force': 'ungrounded',
  'acute renal failure': 'MESH:D058186',
  'acute respiratory failure': 'MESH:D012131',
  'acute rheumatic fever': 'ungrounded',
  'adenosine diphosphate ribosylation factor': 'FPLX:ARF_GTPase_family',
  'adp ribosylation factor': 'FPLX:ARF_GTPase_family',
  'adpribosylation factor': 'FPLX:ARF_GTPase_family',
  'adventitious root formation': 'ungrounded',
  'alkaloid rich fraction': 'ungrounded',
  'alternative open reading frame': 'ungrounded',
  'alternative reading frame': 'ungrounded',
  'alternative reading frame protein': 'ungrounded',
  'anthocyanin rich fraction': 'ungrounded',
  'arf': 'ungrounded',
  'auxin response factor': 'IP:IPR010525'},
 {'MESH:D058186': 'Acute Kidney Injury',
  'MESH:D012131': 'Respiratory Insufficiency',
  'FPLX:ARF_GTPase_family': 'ARF_GTPase_family',
  'IP:IPR010525': 'Auxin response factor'},
 ['FPLX:ARF_GTPase_family', 'IP:IPR010525']]

In [11]:
grounding_map, names, pos_labels = [{'acoustic radiation force': 'ungrounded',
  'acute renal failure': 'MESH:D058186',
  'acute respiratory failure': 'MESH:D012131',
  'acute rheumatic fever': 'ungrounded',
  'adenosine diphosphate ribosylation factor': 'FPLX:ARF_GTPase_family',
  'adp ribosylation factor': 'FPLX:ARF_GTPase_family',
  'adpribosylation factor': 'FPLX:ARF_GTPase_family',
  'adventitious root formation': 'ungrounded',
  'alkaloid rich fraction': 'ungrounded',
  'alternative open reading frame': 'ungrounded',
  'alternative reading frame': 'ungrounded',
  'alternative reading frame protein': 'ungrounded',
  'anthocyanin rich fraction': 'ungrounded',
  'arf': 'ungrounded',
  'auxin response factor': 'IP:IPR010525'},
 {'MESH:D058186': 'Acute Kidney Injury',
  'MESH:D012131': 'Respiratory Insufficiency',
  'FPLX:ARF_GTPase_family': 'ARF_GTPase_family',
  'IP:IPR010525': 'Auxin response factor'},
 ['FPLX:ARF_GTPase_family', 'IP:IPR010525']]

In [12]:
excluded_longforms = ['arf']

In [13]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [14]:
additional_entities = {}

In [16]:
unambiguous_agent_texts = {}

In [17]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [18]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [19]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [20]:
intersection1

[]

In [21]:
intersection2

[]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [23]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-11 05:10:58] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-11 05:11:12] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9772730540464588 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [24]:
classifier.stats

{'label_distribution': {'MESH:D058186': 490,
  'ungrounded': 142,
  'FPLX:ARF_GTPase_family': 225,
  'MESH:D012131': 74,
  'IP:IPR010525': 102},
 'f1': {'mean': 0.977273, 'std': 0.008436},
 'precision': {'mean': 0.976364, 'std': 0.00718},
 'recall': {'mean': 0.978695, 'std': 0.015415},
 'IP:IPR010525': {'f1': {'mean': 0.971196, 'std': 0.017758},
  'pr': {'mean': 0.980952, 'std': 0.023328},
  'rc': {'mean': 0.962771, 'std': 0.034262}},
 'FPLX:ARF_GTPase_family': {'f1': {'mean': 0.980023, 'std': 0.016244},
  'pr': {'mean': 0.977778, 'std': 0.014055},
  'rc': {'mean': 0.982415, 'std': 0.021539}},
 'MESH:D012131': {'f1': {'mean': 0.921177, 'std': 0.028612},
  'pr': {'mean': 0.877143, 'std': 0.054577},
  'rc': {'mean': 0.975, 'std': 0.05}},
 'ungrounded': {'f1': {'mean': 0.93789, 'std': 0.028942},
  'pr': {'mean': 0.90197, 'std': 0.039644},
  'rc': {'mean': 0.977778, 'std': 0.02963}},
 'MESH:D058186': {'f1': {'mean': 0.974923, 'std': 0.006353},
  'pr': {'mean': 0.991837, 'std': 0.009998},
 

In [25]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [26]:
disamb.dump(model_name, results_path)

In [27]:
print(disamb.info())

Disambiguation model for ARF

Produces the disambiguations:
	ARF_GTPase_family*	FPLX:ARF_GTPase_family
	Acute Kidney Injury	MESH:D058186
	Auxin response factor*	IP:IPR010525
	Respiratory Insufficiency	MESH:D012131

Class level metrics:
--------------------
Grounding                	Count	F1     
      Acute Kidney Injury	490	0.97492
        ARF_GTPase_family*	225	0.98002
               Ungrounded	142	0.93789
    Auxin response factor*	102	 0.9712
Respiratory Insufficiency	 74	0.92118

Weighted Metrics:
-----------------
	F1 score:	0.97727
	Precision:	0.97636
	Recall:		0.9787

* Positive labels
See Docstring for explanation



In [28]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1